### Load the dataset, perform all necessary data exploration and preprocessing

## Solution

In [ ]:
import numpy as np
from collections import Counter
from scipy import stats
from math import log2, sqrt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
import seaborn as sns
from sklearn import metrics
from sklearn import tree
from sklearn import preprocessing
import matplotlib.pyplot as plt
import six

from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
from tqdm.notebook import tqdm_notebook as tqdm
from sklearn.model_selection import GridSearchCV

In [ ]:
data=pd.read_csv("../input/vehicle-dataset-from-cardekho/CAR DETAILS FROM CAR DEKHO.csv")
print(data.head())
print(data.shape)

In [ ]:
data=pd.read_csv("../input/vehicle-dataset-from-cardekho/Car details v3.csv")
print(data.head())
print(data.shape)

In [ ]:

data=pd.read_csv("../input/vehicle-dataset-from-cardekho/car data.csv")
print(data.head())
print(data.shape)

**From the abouf data I think "Car details v3.csv" is the most important one.<br> 
As it has many features that have high propability of affecting the price of the care**

In [ ]:
data=pd.read_csv("../input/vehicle-dataset-from-cardekho/Car details v3.csv")
# high-level overview of data shape and composition
print(data.shape)
print(data.dtypes)
print(data.head(10))

### What is the structure of your dataset?

>**Tha data consists of 8128 Raws and 13 columns. <br>**
>>**name(object): the name of the car owner <br>
>>year(Numerical): the year of buying the car <br>
>>selling_price(Numerical): the price of the car. <br>
>>km_driven(Numerical):  the number of km the car has been driven<br>
>>fuel(object): the type of the fuel<br>
>>seller_type(object) <br>
>>transmission(object): wether it automatic or manual <br>
>>owner(object):   the current owner order(if the car is usable then it has been sold to another owner.)<br>
>>mileage(object):a number of miles travelled <br>
>>engine(object): engine number <br>
>>max_power(object): the max power of the car<br>
>>torque(object)<br>
>>seats(Numerical)<br>**



### What is/are the main feature(s) of interest in your dataset?

> I'm most interested in figuring out what features are best for predicting the price of the car in the dataset.
### What features in the dataset do you think will help support your investigation into your feature(s) of interest?
> I think All the feaures will affect the price exept name.Moreover, I think km_driven and mileage are the same just in differnt unit

In [ ]:
def clean_ds(dataset):
    #Drop all duplicates from the dataset
    dataset.drop_duplicates( inplace=True)
    #Drop all null values from the dataset
    dataset.dropna(axis=0,inplace=True)

In [ ]:
print("Data Before cleaning",data.shape)
clean_ds(data)
print("Data After cleaning",data.shape)

**It looks like there were many duplicates and null values**

In [ ]:
data.head()

In [ ]:
print(data.dtypes)

**As we can see that there are some data that we can convert to int because it reflect to a numerical value not an object those attributes are mileage, engine and max_power and torque**

In [ ]:
df=data.copy()
df

In [ ]:
df['mileage'] = df['mileage'].str[:-5].astype(float)
df['engine'] = df['engine'].str[:-2].astype(float)
df['max_power'] = df['max_power'].str[:-3].astype(float)
df['torque'] = df['torque'].str.extract(r'(\d+)')

In [ ]:
df.head()

In [ ]:
df["mileage"]=df["mileage"].astype(int)
df["engine"]=df["engine"].astype(int)
df["max_power"]=df["max_power"].astype(int)
df["torque"]=df["torque"].astype(int)
df["seats"]=df["seats"].astype(int)

In [ ]:
print(df.dtypes)


**Now the data is clean and looks good**

In [ ]:
categorical=["name","fuel","seller_type","transmission","owner","mileage","engine","max_power","torque"]
numerical=["seats","km_driven","year"]


In [ ]:
def ca_to_num(ds,at):
    ds[at] = ds[at].astype('category')
    ds[at] = ds[at].cat.codes

In [ ]:
for i in categorical:
    ca_to_num(df,i)

In [ ]:
#List contain all the columns names
d=list(df.columns)
# drop the selling_price from the colums name and consider the list as the features
# and the l is the label
l=d.pop(2)
a = df[d]
b=df[l]


In [ ]:
df.dtypes

## 3 Points: Use Sklearn Random Forest Regressor as the ML model

### Perform proper hyperparameter tuning to choose the best base model, depth, max_features, and any other important hyper-parameters 

## Solution

In [ ]:
#List contain all the columns names
d=list(df.columns)
# drop the selling_price from the colums name and consider the list as the features
# and the l is the label
l=d.pop(2)
X = df[d]
y=df[l]
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 10)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RandomizedSearchCV
n_estimators= [1,5,10,30,50]
max_features= ["auto", "sqrt", "log2"]
max_depth= [4,10,20,12,3,9,None]
min_samples_split=[3, 5, 6,10,20,40]
min_samples_leaf=[5, 2 ,6,20,3,12,7,10]
hp ={'n_estimators': n_estimators,"criterion":['mse','mae'], 'max_features': max_features,"max_depth":max_depth, 
 'min_samples_leaf':min_samples_leaf, 'min_samples_split':min_samples_split}

clsif=RandomForestRegressor(random_state=1)
RFR_random = RandomizedSearchCV(clsif, hp, n_iter = 10, cv = 5, verbose=2, random_state=10, n_jobs = -1) 
RFR_random.fit(X_train,y_train)
print(f"Best result {RFR_random.best_params_}")

In [ ]:
clf2 = RandomForestRegressor(n_estimators = 10,min_samples_split=6,min_samples_leaf=5 ,max_features="auto",max_depth=None, criterion = 'mae')
clf2.fit(X_train, y_train)
y_pred = clf2.predict(X_test)
print("Accuracy:",clf2.score(X_test, y_test))

## 4 Points: Use Sklearn AdaBoost on the same dataset and compare the results with the RF

## Solution

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

param_dist = {
 'n_estimators': n_estimators,
 'learning_rate' : [0.01,0.05,0.1,0.4,0.2,0.3,0.5,0.6,0.8,1],
 'loss' : ['linear', 'square', 'exponential']
 }
hyper = RandomizedSearchCV(AdaBoostRegressor(), param_distributions = param_dist,cv=3,n_iter = 10,n_jobs=-1)
hyper.fit(X_train, y_train)
hyper.best_params_

In [ ]:
#List contain all the columns names
d=list(df.columns)
# drop the selling_price from the colums name and consider the list as the features
# and the l is the label
l=d.pop(2)
X = df[d]
y=df[l]
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 10)


In [ ]:
from sklearn.ensemble import AdaBoostRegressor
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 10)
clf2 = AdaBoostRegressor(n_estimators = 50,loss='linear',learning_rate=0.2)
clf2.fit(X_train, y_train)
y_pred = clf2.predict(X_test)
print("Accuracy:",clf2.score(X_test, y_test))

**The accuracy of the randomforestregressor is higher than adaboostregressor**

## Bonus Task ( 3 Points ) : Use another Boosting Algorithm, such as XGBoost, on the same dataset. 

### You need to explain in details how the algorithm work and how it is different from AdaBoost in order to receive any credit.

## Solution

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb_model = XGBRegressor(learning_rate=0.11,max_depth=150,n_estimators=150)
xgb_model.fit(X_train,y_train)
y_pred = xgb_model.predict(X_test)
print("Accuracy:",xgb_model.score(X_test, y_test))

**There is a diffinition called similarity or quality score.<br>
It's calculated by squere (the summing of  the residual values) and then divide the result by (thier number + learning rate) <br>**

**Residuals is the differce between the observed and the pridected value**

**gain= left branch similarity +right branch similarity -root similarity**

* At first the XGBoost start from a single leaf of the tree.
* Then start with intial pridiction with any value the diffult is 0.5
* Calculate the resdual and but the values in the single leaf
* Calculate the quality score or the similarity 
* Then we try to do better by spliting the residuals into groups.
* Here we try to find the best split to reduce the residuals and split them into two groups
* Then we will have two branch one on the left another on the right
* we calculate the similarity score for both branches
* then we need to quantify how much better the leaves cluster similar residuals than the root by calculating the gain
* then we go into the residuals again and split them and calculate the gain 
*  and then compare the gain again and see with is better 
* we do this till find the best fit or reaching the asked spliting number